Opens the Bursa Malaysia website (Indices page).

Finds the market share table

Goes through pagination (if multiple pages)

Extracts data from the HTML table(s)

Appends each page’s data into a DataFrame

Exports the result as a CSV

In [2]:
!pip install selenium
!pip install webdriver-manager


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from io import StringIO
from datetime import datetime
from bs4 import BeautifulSoup
import os
import time


In [9]:
try:
    #Chrome
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


    driver.get("https://www.bursamarketplace.com/mkt/themarket/stock")
    driver.save_screenshot("debug_Market_Share.png")

    #no iframe on this page,removed in recent bursa redesign
    #Wait for iframe and switch to it
    '''
    WebDriverWait(driver, 15).until(EC.frame_to_be_available_and_switch_to_it((By.TAG_NAME, "iframe")))
    '''

    #Wait for table rows rendered (instead of only table tag)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, ".tb_row.tb_data"))
    )
    headers = [
        "CASHTAG", "NAME", "PRICE", "CLOSE", "CHG", "%CHG",
        "VOLUME", "HIGH", "LOW", "BUY (QTY)", "SELL (QTY)", "PAGE"
    ]

    
    '''
    #html
    table = driver.find_element(By.TAG_NAME, "table")
    html = table.get_attribute("outerHTML")
    '''

   # wait until the data-total attribute is populated.
    def wait_for_total_pages(driver, timeout=10):
        end_time = time.time() + timeout
        while time.time() < end_time:
            try:
                elem = driver.find_element(By.CSS_SELECTOR, "[name='listing_paging_total']")
                total_text = elem.get_attribute("data-total")
                if total_text and total_text.isdigit():
                    return int(total_text)
            except:
                pass
            time.sleep(0.5)
        raise Exception("Timeout: Could not read total page count from data-total attribute")

    
    all_rows = []
    total_pages = wait_for_total_pages(driver)
    for page in range(1, total_pages + 1):
        print(f"Scraping page {page}/{total_pages}")
    
        # Trigger the JS function manually using execute_script
        js_cmd = f"callListAjax('stock_ajax','listing',{page}, 'name', 'no');"
        driver.execute_script(js_cmd)
    
        # Wait for the page to update (wait for a specific row or CASHTAG change)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.tb_row.tb_data"))
        )
    
        # Parse table rows using BeautifulSoup
        soup = BeautifulSoup(driver.page_source, "html.parser")
        for row in soup.select("div.tb_row.tb_data"):
            cols = []
            for cell in row.select("div.tb_cell"):
                link = cell.find("a")
                text = link.text.strip() if link else cell.get_text(strip=True)
                cols.append(text)
            if cols:
                cols.append(page)  # Append the current page number
                all_rows.append(cols)


            
    df = pd.DataFrame(all_rows, columns=headers[:len(all_rows[0])])
    print(df.head()) 
    print(f"Rows found: {len(df)}")
    if len(df) == 0:
        print("need more time for data to laod,maybe")


    df.index.name = "No"  

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    #check data folder
    os.makedirs("data", exist_ok=True)
    #filename = f"bursa_market_share{timestamp}.csv"
    filename = os.path.join("data", f"bursa_market_share.csv")

    df.to_csv(filename, index=True)
    #df.to_csv("bursa_indices.csv", index=False)
    print("done save file : 'bursa_market_share.csv'")

except Exception as e:
    print("error", e)

finally:
    # Only quit if the driver exists,in real life just give up
    try:
        driver.quit()
    except:
        pass

error Message: 
Stacktrace:
	GetHandleVerifier [0x0x9744a3+62419]
	GetHandleVerifier [0x0x9744e4+62484]
	(No symbol) [0x0x7b2133]
	(No symbol) [0x0x7fa8fe]
	(No symbol) [0x0x7fac9b]
	(No symbol) [0x0x843052]
	(No symbol) [0x0x81f4b4]
	(No symbol) [0x0x84087a]
	(No symbol) [0x0x81f266]
	(No symbol) [0x0x7ee852]
	(No symbol) [0x0x7ef6f4]
	GetHandleVerifier [0x0xbe4793+2619075]
	GetHandleVerifier [0x0xbdfbaa+2599642]
	GetHandleVerifier [0x0x99b04a+221050]
	GetHandleVerifier [0x0x98b2c8+156152]
	GetHandleVerifier [0x0x991c7d+183213]
	GetHandleVerifier [0x0x97c388+94904]
	GetHandleVerifier [0x0x97c512+95298]
	GetHandleVerifier [0x0x96766a+9626]
	BaseThreadInitThunk [0x0x74df7ba9+25]
	RtlInitializeExceptionChain [0x0x76f2c3ab+107]
	RtlClearBits [0x0x76f2c32f+191]

